In [1]:
## Programa para descargar listado de ofertas en buscalibre
import requests
from lxml import html
import pandas as pd

In [2]:
## Evitar problemas de bloqueos ip
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'
}

In [5]:
## BD vacia
bd_final = pd.DataFrame()

In [6]:
## Loop en la paginación de buscalibre
# Si no hay data, termina el loop

for x in range(1,100):
    url = "https://www.buscalibre.pe/libros/search?q=yayas&page=" + str(x)
    req = requests.get(url, headers = headers)
    parseo = html.fromstring(req.content)

    titulos =     parseo.xpath("//div[@class = 'productos pais160']/div//h3/text()")
    autores =     parseo.xpath("//div[@class = 'productos pais160']/div//div[@class = 'autor']/text()")
    editoriales = parseo.xpath("//div[@class = 'productos pais160']/div//div[@class = 'autor color-dark-gray metas hide-on-hover']/text()")
    descuentos =  parseo.xpath("//div[@class = 'productos pais160']/div//div[@class = 'descuento-v2 color-white position-relative']/text()")
    precios =     parseo.xpath("//div[@class = 'productos pais160']/div//p[@class = 'precio-ahora hide-on-hover margin-0 font-size-medium']/strong/text()")

    precios = [precio.replace('S/  ', '').replace(',','.') for precio in precios]
    editoriales = [edit.replace('\n                                                ','').replace('\n                                        ','') for edit in editoriales]
    
    if titulos != []:
        data = pd.DataFrame(list(zip(titulos,autores,editoriales,descuentos,precios)), columns = ['titulos','autores','editoriales','descuentos','precios'])
        bd_final = bd_final.append(data, ignore_index=True)
    else:
        break


In [7]:
bd_final

,titulos,autores,editoriales,descuentos,precios
0,(Yayas) Columpios,Alberto Forcada,"Fondo De Cultura Económica, 2005, Tapa dura, N...",60%,35.66
1,(Yayas)Esa Mañana,Toño Malpica,"Norma, 1ª edición, Tapa Blanda, Nuevo",60%,40.56
2,(Yayas)Majestic 2,"Dan Abnett,Andy Lanning","Norma Editorial, S.A., Tapa Blanda, Nuevo",60%,25.79
3,(Yayas) Juanes,Diego Londoño,"Aguilar, Tapa Blanda, Nuevo",60%,54.68
4,(Yayas)Chubasco,"Latini, Cielo","Booket, Tapa Blanda, Nuevo",60%,49.57
...,...,...,...,...,...
995,La Importancia de la i+d en Situaciones de Cr...,Miriam Julia Mate Lord&Eacute;N; Jos&Eacute; M...,"Editorial Académica Española, 2021, Tapa bland...",50%,456.22
996,zumbas con que el famoso juan de espera en di...,joseph santas de capuano,"nabu press, Nuevo",40%,233.11
997,Le financement public des entreprises culture...,Yaya Soura,"Presses Académiques Francophones, Tapa Blanda,...",40%,231.72
998,Pesos específicos de factores de riesgo en ba...,José Franco Monsreal,"Editorial Académica Española, Tapa Blanda, Nuevo",40%,246.73


In [8]:
bd_final.to_excel(r"C:\Users\abaldeon\Downloads\BuscaLibre_1.xlsx")